In [ ]:
!pip install ngrok flask bleach

import google.colab.userdata
import ngrok

import flask
import sqlite3
import contextlib
import html
import bleach

app = flask.Flask(__name__)

# データベース用意
conn = sqlite3.connect(":memory:", check_same_thread = False, isolation_level=None, timeout=1.0)
conn.execute('pragma busy_timeout=1000')
conn.executescript(
    """
    CREATE TABLE post(id INTEGER PRIMARY KEY AUTOINCREMENT, name STRING, text STRING);
    INSERT INTO post(name, text) VALUES ("管理人","初めての投稿です");
    """
)

@app.route("/", methods=["GET"])
def root():
        cookie_name = flask.request.cookies.get('name') or ""

        param_name = flask.request.args.get("name") or ""
        param_text = flask.request.args.get("text") or ""

        if param_name != "":
            res =  flask.redirect("/", code=302)
            res.set_cookie("name",param_name)
            return res

        if cookie_name == "":
            response_html = """
            <html>
            <head>
            <meta charset="UTF-8">
            <title>掲示板 : Webアプリセキュリティ演習</title>
            </head>
            <body>
            <h1>ユーザー登録</h1>
            <form>名前: <input type="text" name="name"><br><button type="submit">登録</button></form>
            </body>
            </html>
            """
            return flask.Response(response_html, status=200, mimetype='text/html;charset=UTF-8')

        if param_text != "":
            with contextlib.closing(conn.cursor()) as cur:
                cur.execute("INSERT INTO post(name, text) VALUES (?,?)", (cookie_name, param_text))
            return flask.redirect("/", code=302)


        response_html = """
        <html>
        <head>
        <meta charset="UTF-8">
        <title>掲示板 : Webアプリセキュリティ演習</title>
        </head>
        <body>
        <h1>掲示板</h1>
        """

        with contextlib.closing(conn.cursor()) as cur:
            cur.execute("SELECT name,text FROM post ORDER BY id desc LIMIT 50")
            data = cur.fetchall()
            for d in reversed(data):
                response_html += "<h3>" + html.escape(str(d[0])) + "</h3>"
                response_html += "<div>" + bleach.clean(str(d[1]), tags=["img"], attributes={'img': ['src']}) + "</div>"

        response_html += """
        <h1>新規投稿</h1>
        <form method="get">
            <textarea style="height:200px;width:350px" name="text"></textarea><br>
            <button type="submit">投稿</button><br>
            imgタグのみ使用可能です。
        </form>
        </body>
        </html>
        """
        return flask.Response(response_html, status=200, mimetype='text/html;charset=UTF-8')

# 起動
listener = await ngrok.forward(5000, authtoken=google.colab.userdata.get("ngrok-token"))
print(listener.url())
app.run(host="0.0.0.0", port=5000)



普通の画像

```
<img src="https://placehold.jp/e60000/ffffff/200x50.png?text=%E3%83%93%E3%83%83%E3%82%AF%E3%82%AB%E3%83%A1%E3%83%A9">
```

攻撃できる文字列

```
<img src="/?text=%E5%AE%B6%E9%9B%BB%E3%82%92%E8%B2%B7%E3%81%86%E3%81%AA%E3%82%89%E3%83%93%E3%83%83%E3%82%AF%E3%82%AB%E3%83%A1%E3%83%A9%E3%81%AB%E6%B1%BA%E3%81%BE%E3%81%A3%E3%81%A6%E3%82%8B">
```
